<center><h1>第二章 pandas基础</h1></center>

In [ ]:
import numpy as np
import pandas as pd

在开始学习前，请保证`pandas`的版本号不低于如下所示的版本，否则请务必升级！请确认已经安装了`xlrd, xlwt, openpyxl`这三个包，其中`xlrd`版本不得高于`2.0.0`。

In [ ]:
pd.__version__

## 一、文件的读取和写入
### 1. 文件读取

`pandas`可以读取的文件格式有很多，这里主要介绍读取`csv, excel, txt`文件。

In [ ]:
df_csv = pd.read_csv('../data/my_csv.csv')
df_csv

In [ ]:
df_txt = pd.read_table('../data/my_table.txt')
df_txt

In [ ]:
df_excel = pd.read_excel('../data/my_excel.xlsx')
df_excel

这里有一些常用的公共参数，`header=None`表示第一行不作为列名，`index_col`表示把某一列或几列作为索引，索引的内容将会在第三章进行详述，`usecols`表示读取列的集合，默认读取所有的列，`parse_dates`表示需要转化为时间的列，关于时间序列的有关内容将在第十章讲解，`nrows`表示读取的数据行数。上面这些参数在上述的三个函数里都可以使用。

In [ ]:
pd.read_table('../data/my_table.txt', header=None)

In [ ]:
pd.read_csv('../data/my_csv.csv', index_col=['col1', 'col2'])

In [ ]:
pd.read_table('../data/my_table.txt', usecols=['col1', 'col2'])

In [ ]:
pd.read_csv('../data/my_csv.csv', parse_dates=['col5'])

In [ ]:
pd.read_excel('../data/my_excel.xlsx', nrows=2)

在读取`txt`文件时，经常遇到分隔符非空格的情况，`read_table`有一个分割参数`sep`，它使得用户可以自定义分割符号，进行`txt`数据的读取。例如，下面的读取的表以`||||`为分割：

In [ ]:
pd.read_table('../data/my_table_special_sep.txt')

上面的结果显然不是理想的，这时可以使用`sep`，同时需要指定引擎为`python`：

In [ ]:
pd.read_table('../data/my_table_special_sep.txt', sep=' \|\|\|\| ', engine='python')

In [ ]:
pd.read_table('../data/my_table_special_sep.txt', sep=r' \|\|\|\| ', engine='python')

这里用 raw string 没有效果

#### 【WARNING】`sep`是正则参数

在使用`read_table`的时候需要注意，参数`sep`中使用的是正则表达式，因此需要对`|`进行转义变成`\|`，否则无法读取到正确的结果。有关正则表达式的基本内容可以参考第八章或者其他相关资料。

#### 【END】

### 2. 数据写入

一般在数据写入中，最常用的操作是把`index`设置为`False`，特别当索引没有特殊意义的时候，这样的行为能把索引在保存的时候去除。

In [ ]:
df_csv.to_csv('../data/my_csv_saved.csv', index=False)
df_excel.to_excel('../data/my_excel_saved.xlsx', index=False)

`pandas`中没有定义`to_table`函数，但是`to_csv`可以保存为`txt`文件，并且允许自定义分隔符，常用制表符`\t`分割：

In [ ]:
df_txt.to_csv('../data/my_txt_saved.txt', sep='\t', index=False)

如果想要把表格快速转换为`markdown`和`latex`语言，可以使用`to_markdown`和`to_latex`函数，此处需要安装`tabulate`包。

In [ ]:
print(df_csv.to_markdown())

In [ ]:
result = df_csv.to_latex()
type(result)

In [ ]:
with open('df_csv_to_latex.txt', 'w') as f:
    f.write(result)

In [ ]:
pwd

In [ ]:
print(df_csv.to_latex())

## 二、基本数据结构
`pandas`中具有两种基本的数据存储结构，存储一维`values`的`Series`和存储二维`values`的`DataFrame`，在这两种结构上定义了很多的属性和方法。

### 1. Series
`Series`一般由四个部分组成，分别是序列的值`data`、索引`index`、存储类型`dtype`、序列的名字`name`。其中，索引也可以指定它的名字，默认为空。

In [ ]:
s = pd.Series(data = [100, 'a', {'dic1':5}],
              index = pd.Index(['id1', 20, 'third'], name='my_idx'),
              dtype = 'object',
              name = 'my_name')
s

#### 【NOTE】`object`类型

`object`代表了一种混合类型，正如上面的例子中存储了整数、字符串以及`Python`的字典数据结构。此外，目前`pandas`把纯字符串序列也默认认为是一种`object`类型的序列，但它也可以用`string`类型存储，文本序列的内容会在第八章中讨论。

#### 【END】

对于这些属性，可以通过 . 的方式来获取：

In [ ]:
s.values

In [ ]:
s.index

In [ ]:
s.dtype

In [ ]:
s.name

利用`.shape`可以获取序列的长度：

In [ ]:
s.shape

索引是`pandas`中最重要的概念之一，它将在第三章中被详细地讨论。如果想要取出单个索引对应的值，可以通过`[index_item]`可以取出。

### 2. DataFrame
`DataFrame`在`Series`的基础上增加了列索引，一个数据框可以由二维的`data`与行列索引来构造：

In [ ]:
data = [[1, 'a', 1.2], [2, 'b', 2.2], [3, 'c', 3.2]]
df = pd.DataFrame(data = data,
                  index = ['row_%d'%i for i in range(3)],
                  columns=['col_0', 'col_1', 'col_2'])
df

但一般而言，更多的时候会采用从列索引名到数据的映射来构造数据框，同时再加上行索引：

In [ ]:
list('abc')

In [ ]:
df = pd.DataFrame(data = {'col_0': [1,2,3],
                          'col_1':list('abc'),
                          'col_2': [1.2, 2.2, 3.2]},
                  index = ['row_%d'%i for i in range(3)])
df

由于这种映射关系，在`DataFrame`中可以用`[col_name]`与`[col_list]`来取出相应的列与由多个列组成的表，结果分别为`Series`和`DataFrame`：

In [ ]:
df['col_0']

In [ ]:
df[['col_0', 'col_1']]

与`Series`类似，在数据框中同样可以取出相应的属性：

In [ ]:
df.values

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.dtypes # 返回的是值为相应列数据类型的Series

In [ ]:
df.shape

通过`.T`可以把`DataFrame`进行转置：

In [ ]:
df.T

## 三、常用基本函数
为了进行举例说明，在接下来的部分和其余章节都将会使用一份`learn_pandas.csv`的虚拟数据集，它记录了四所学校学生的体测个人信息。

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/learn_pandas.csv')
df.columns

In [ ]:
df

上述列名依次代表学校、年级、姓名、性别、身高、体重、是否为转系生、体测场次、测试时间、1000米成绩，本章只需使用其中的前七列。

In [ ]:
df = df[df.columns[:7]]

In [ ]:
df

### 1. 汇总函数
`head, tail`函数分别表示返回表或者序列的前`n`行和后`n`行，其中`n`默认为5：

In [ ]:
df.head(2)

In [ ]:
df.tail(3)

`info, describe`分别返回表的信息概况和表中数值列对应的主要统计量 ：

In [ ]:
df.info()

In [ ]:
df.describe()

#### 【NOTE】更全面的数据汇总

`info, describe`只能实现较少信息的展示，如果想要对一份数据集进行全面且有效的观察，特别是在列较多的情况下，推荐使用[pandas-profiling](https://pandas-profiling.github.io/pandas-profiling/docs/)包，它将在第十一章被再次提到。

#### 【END】

### 2. 特征统计函数
在`Series`和`DataFrame`上定义了许多统计函数，最常见的是`sum, mean, median, var, std, max, min`。例如，选出身高和体重列进行演示：

In [ ]:
df_demo = df[['Height', 'Weight']]
df_demo.mean()

In [ ]:
df_demo.max()

此外，需要介绍的是`quantile, count, idxmax`这三个函数，它们分别返回的是分位数、非缺失值个数、最大值对应的索引：

In [ ]:
df_demo.quantile(0.75)

In [ ]:
df_demo.count()

In [ ]:
df_demo.idxmax() # idxmin是对应的函数

In [ ]:
df_demo

上面这些所有的函数，由于操作后返回的是标量，所以又称为聚合函数，它们有一个公共参数`axis`，默认为0代表逐列聚合，如果设置为1则表示逐行聚合：

In [ ]:
df_demo.mean(axis=1).head() # 在这个数据集上体重和身高的均值并没有意义

In [ ]:
df_demo.mean(axis=0).head() # 在这个数据集上体重和身高的均值并没有意义

### 3. 唯一值函数
对序列使用`unique`和`nunique`可以分别得到其唯一值组成的列表和唯一值的个数：

In [ ]:
df['School']

In [ ]:
df['School'].unique()

In [ ]:
df['School'].nunique()

`value_counts`可以得到唯一值和其对应出现的频数：

In [ ]:
df['School'].value_counts()

In [ ]:
df['Name'].value_counts()

如果想要观察多个列组合的唯一值，可以使用`drop_duplicates`。其中的关键参数是`keep`，默认值`first`表示每个组合保留第一次出现的所在行，`last`表示保留最后一次出现的所在行，`False`表示把所有重复组合所在的行剔除。

In [ ]:
df_demo = df[['Gender','Transfer','Name']]
df_demo

In [ ]:
df_demo.drop_duplicates(['Gender', 'Transfer'])

In [ ]:
df_demo.drop_duplicates(['Gender', 'Transfer'], keep='last')

In [ ]:
df_demo

In [ ]:
df_demo.drop_duplicates(['Name', 'Gender'], keep=False).head() # 保留只出现过一次的性别和姓名组合

In [ ]:
df_demo.drop_duplicates(['Name', 'Gender'], keep=False)

In [ ]:
df_demo.drop_duplicates(['Name', 'Gender'], keep='first')

In [ ]:
df_demo.drop_duplicates(['Name', 'Gender'], keep='last')

In [ ]:
df['School']

In [ ]:
type(df['School'])

In [ ]:
df['School'].drop_duplicates() # 在Series上也可以使用

In [ ]:
df['Name']

In [ ]:
df['Name'].drop_duplicates()

In [ ]:
df['Name'].drop_duplicates(keep=False)

此外，`duplicated`和`drop_duplicates`的功能类似，但前者返回了是否为唯一值的布尔列表，其`keep`参数与后者一致。其返回的序列，把重复元素设为`True`，否则为`False`。 `drop_duplicates`等价于把`duplicated`为`True`的对应行剔除。

In [ ]:
df_demo.duplicated(['Gender', 'Transfer']).head()

In [ ]:
df_demo.duplicated(['Gender', 'Transfer'])

In [ ]:
df_demo.drop_duplicates(['Gender', 'Transfer'])

In [ ]:
df['School'].duplicated().head() # 在Series上也可以使用

### 4. 替换函数
一般而言，替换操作是针对某一个列进行的，因此下面的例子都以`Series`举例。`pandas`中的替换函数可以归纳为三类：映射替换、逻辑替换、数值替换。其中映射替换包含`replace`方法、第八章中的`str.replace`方法以及第九章中的`cat.codes`方法，此处介绍`replace`的用法。

在`replace`中，可以通过字典构造，或者传入两个列表来进行替换：

In [ ]:
df

In [ ]:
df['Gender'].replace({'Female':0, 'Male':1}).head()

In [ ]:
df['Gender'].replace(['Female', 'Male'], [0, 1]).head()

另外，`replace`还有一种特殊的方向替换，指定`method`参数为`ffill`则为用前面一个最近的未被替换的值进行替换，`bfill`则使用后面最近的未被替换的值进行替换。从下面的例子可以看到，它们的结果是不同的：

In [ ]:
s = pd.Series(['a', 1, 'b', 2, 1, 1, 'a'])
s.replace([1, 2], method='ffill')

In [ ]:
s.replace([1, 2], method='bfill')

#### 【WARNING】正则替换请使用`str.replace`

虽然对于`replace`而言可以使用正则替换，但是当前版本下对于`string`类型的正则替换还存在`bug`，因此如有此需求，请选择`str.replace`进行替换操作，具体的方式将在第八章中讲解。

#### 【END】

逻辑替换包括了`where`和`mask`，这两个函数是完全对称的：`where`函数在传入条件为`False`的对应行进行替换，而`mask`在传入条件为`True`的对应行进行替换，当不指定替换值时，替换为缺失值。

In [ ]:
s = pd.Series([-1, 1.2345, 100, -50])
s.where(s<0)

In [ ]:
s.where(s<0, 100)

In [ ]:
s.mask(s<0)

In [ ]:
s.mask(s<0, -50)

需要注意的是，传入的条件只需是与被调用的`Series`索引一致的布尔序列即可：

In [ ]:
s_condition= pd.Series([True,False,False,True],index=s.index)
s.mask(s_condition, -50)

数值替换包含了`round, abs, clip`方法，它们分别表示按照给定精度四舍五入、取绝对值和截断：

In [ ]:
s = pd.Series([-1, 1.2345, 100, -50])
s.round(2)

In [ ]:
s.abs()

In [ ]:
s.clip(0, 2) # 前两个数分别表示上下截断边界

#### 【练一练】

在 clip 中，超过边界的只能截断为边界值，如果要把超出边界的替换为自定义的值，应当如何做？

In [ ]:
t = s.clip(0, 2)
t.mask(t==0.0, 'low')
t.mask(t==2, 'large')

In [ ]:
t[0] == 0.0

#### 【END】

### 5. 排序函数
排序共有两种方式，其一为值排序，其二为索引排序，对应的函数是`sort_values`和`sort_index`。

为了演示排序函数，下面先利用`set_index`方法把年级和姓名两列作为索引，多级索引的内容和索引设置的方法将在第三章进行详细讲解。

In [ ]:
df_demo = df[['Grade', 'Name', 'Height', 'Weight']].set_index(['Grade','Name'])
df_demo.head(3)

In [ ]:
df_demo

对身高进行排序，默认参数`ascending=True`为升序：

In [ ]:
df_demo.sort_values('Height').head()

In [ ]:
df_demo.sort_values('Height', ascending=False).head()

在排序中，经常遇到多列排序的问题，比如在体重相同的情况下，对身高进行排序，并且保持身高降序排列，体重升序排列：

In [ ]:
df_demo.sort_values(['Weight','Height'],ascending=[True,False]).head()

索引排序的用法和值排序完全一致，只不过元素的值在索引中，此时需要指定索引层的名字或者层号，用参数`level`表示。另外，需要注意的是字符串的排列顺序由字母顺序决定。

In [ ]:
df_demo.sort_index(level=['Grade','Name'],ascending=[True,False]).head()

In [ ]:
df_demo.sort_index(level=['Grade','Name'],ascending=[True,False])

### 6. apply方法
`apply`方法常用于`DataFrame`的行迭代或者列迭代，它的`axis`含义与第2小节中的统计聚合函数一致，`apply`的参数往往是一个以序列为输入的函数。例如对于`.mean()`，使用`apply`可以如下地写出：

In [ ]:
df_demo = df[['Height', 'Weight']]
def my_mean(x):
    res = x.mean()
    return res
df_demo.apply(my_mean)

同样的，可以利用`lambda`表达式使得书写简洁，这里的`x`就指代被调用的`df_demo`表中逐个输入的序列：

In [ ]:
df_demo.apply(lambda x:x.mean())

若指定`axis=1`，那么每次传入函数的就是行元素组成的`Series`，其结果与之前的逐行均值结果一致。

In [ ]:
df_demo.apply(lambda x:x.mean(), axis=1).head()

这里再举一个例子：`mad`函数返回的是一个序列中偏离该序列均值的绝对值大小的均值，例如序列1,3,7,10中，均值为5.25，每一个元素偏离的绝对值为4.25,2.25,1.75,4.75，这个偏离序列的均值为3.25。现在利用`apply`计算升高和体重的`mad`指标：

In [ ]:
df_demo.apply(lambda x:(x-x.mean()).abs().mean())

这与使用内置的`mad`函数计算结果一致：

In [ ]:
df_demo.mad()

#### 【WARNING】谨慎使用`apply`

得益于传入自定义函数的处理，`apply`的自由度很高，但这是以性能为代价的。一般而言，使用`pandas`的内置函数处理和`apply`来处理同一个任务，其速度会相差较多，因此只有在确实存在自定义需求的情境下才考虑使用`apply`。

#### 【END】

## 四、窗口对象
`pandas`中有3类窗口，分别是滑动窗口`rolling`、扩张窗口`expanding`以及指数加权窗口`ewm`。需要说明的是，以日期偏置为窗口大小的滑动窗口将在第十章讨论，指数加权窗口见本章练习。

### 1. 滑窗对象
要使用滑窗函数，就必须先要对一个序列使用`.rolling`得到滑窗对象，其最重要的参数为窗口大小`window`。

In [ ]:
s = pd.Series([1,2,3,4,5])
roller = s.rolling(window = 3)
roller

在得到了滑窗对象后，能够使用相应的聚合函数进行计算，需要注意的是窗口包含当前行所在的元素，例如在第四个位置进行均值运算时，应当计算(2+3+4)/3，而不是(1+2+3)/3：

In [ ]:
roller.mean()

In [ ]:
roller.sum()

对于滑动相关系数或滑动协方差的计算，可以如下写出：

In [ ]:
s2 = pd.Series([1,2,6,16,30])
roller.cov(s2)

In [ ]:
roller.corr(s2)

此外，还支持使用`apply`传入自定义函数，其传入值是对应窗口的`Series`，例如上述的均值函数可以等效表示：

In [ ]:
roller.apply(lambda x:x.mean())

`shift, diff, pct_change`是一组类滑窗函数，它们的公共参数为`periods=n`，默认为1，分别表示取向前第`n`个元素的值、与向前第`n`个元素做差（与`Numpy`中不同，后者表示`n`阶差分）、与向前第`n`个元素相比计算增长率。这里的`n`可以为负，表示反方向的类似操作。

In [ ]:
s = pd.Series([1,3,6,10,15])
s.shift(2)

In [ ]:
s.diff(3)

In [ ]:
s.pct_change()

In [ ]:
s.shift(-1)

In [ ]:
s.diff(-2)

In [ ]:
s

In [ ]:
import numpy as np
np.diff(s)

In [ ]:
np.diff(s, n=2)

将其视作类滑窗函数的原因是，它们的功能可以用窗口大小为`n+1`的`rolling`方法等价代替：

In [ ]:
s.shift(2)

In [ ]:
s

In [ ]:
s.rolling(3).apply(lambda x:list(x)[0]) # s.shift(2)

In [ ]:
s.rolling(3).apply(lambda x:list(x)[2])

In [ ]:
s.diff(3)

In [ ]:
 s.rolling(4).apply(lambda x:list(x)[-1]-list(x)[0]) # s.diff(3)

In [ ]:
def my_pct(x):
    L = list(x)
    return L[-1]/L[0]-1
s.rolling(2).apply(my_pct) # s.pct_change()

#### 【练一练】

`rolling`对象的默认窗口方向都是向前的，某些情况下用户需要向后的窗口，例如对1,2,3设定向后窗口为2的`sum`操作，结果为3,5,NaN，此时应该如何实现向后的滑窗操作？

> 先将数组反转，进行 rolling 操作，再将结果反转。

In [ ]:
s = pd.Series([1, 2, 3])
s

In [ ]:
s2 = s.sort_index(ascending=False)

In [ ]:
rol = s2.rolling(2)

In [ ]:
rol.mean()

In [ ]:
rol.sum()

#### 【END】

### 2. 扩张窗口
扩张窗口又称累计窗口，可以理解为一个动态长度的窗口，其窗口的大小就是从序列开始处到具体操作的对应位置，其使用的聚合函数会作用于这些逐步扩张的窗口上。具体地说，设序列为a1, a2, a3, a4，则其每个位置对应的窗口即\[a1\]、\[a1, a2\]、\[a1, a2, a3\]、\[a1, a2, a3, a4\]。

In [2]:
import pandas as pd
import numpy as np

In [3]:
s = pd.Series([1, 3, 6, 10])
s.expanding().mean()

0    1.000000
1    2.000000
2    3.333333
3    5.000000
dtype: float64

#### 【练一练】

`cummax, cumsum, cumprod`函数是典型的类扩张窗口函数，请使用`expanding`对象依次实现它们。


In [4]:
s.cummax()

0     1
1     3
2     6
3    10
dtype: int64

In [7]:
s.expanding().max()

0     1.0
1     3.0
2     6.0
3    10.0
dtype: float64

In [5]:
s.cumsum()

0     1
1     4
2    10
3    20
dtype: int64

In [8]:
s.expanding().sum()

0     1.0
1     4.0
2    10.0
3    20.0
dtype: float64

In [14]:
s.cumprod()

0      1
1      3
2     18
3    180
dtype: int64

In [16]:
def prod(s):
    # 输入pandas series
    r = 1
    for num in s:
        r *= num
    return r

In [17]:
s.expanding().apply(prod)

0      1.0
1      3.0
2     18.0
3    180.0
dtype: float64

#### 【END】

## 五、练习
### Ex1：口袋妖怪数据集
现有一份口袋妖怪的数据集，下面进行一些背景说明：

* `#`代表全国图鉴编号，不同行存在相同数字则表示为该妖怪的不同状态

* 妖怪具有单属性和双属性两种，对于单属性的妖怪，`Type 2`为缺失值
* `Total, HP, Attack, Defense, Sp. Atk, Sp. Def, Speed`分别代表种族值、体力、物攻、防御、特攻、特防、速度，其中种族值为后6项之和

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/pokemon.csv')
df.head(3)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80


In [3]:
df

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80


**1** 对`HP, Attack, Defense, Sp. Atk, Sp. Def, Speed`进行加总，验证是否为`Total`值。

In [4]:
t1 = df['Total']
t2 = df.iloc[:, 5:]

In [5]:
t2

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,45,49,49,65,65,45
1,60,62,63,80,80,60
2,80,82,83,100,100,80
3,80,100,123,122,120,80
4,39,52,43,60,50,65
...,...,...,...,...,...,...
795,50,100,150,100,150,50
796,50,160,110,160,110,110
797,80,110,60,150,130,70
798,80,160,60,170,130,80


In [6]:
False in (t2.sum(1) == t1)

False

**2** 对于`#`重复的妖怪只保留第一条记录，解决以下问题：

* 求第一属性的种类数量和前三多数量对应的种类
* 求第一属性和第二属性的组合种类
* 求尚未出现过的属性组合

In [7]:
df2 = df.drop_duplicates(['#'], keep='first')  # 去重
df2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80
...,...,...,...,...,...,...,...,...,...,...,...
793,717,Yveltal,Dark,Flying,680,126,131,95,131,98,99
794,718,Zygarde50% Forme,Dragon,Ground,600,108,100,121,81,95,95
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70


In [8]:
df2['Type 1'].nunique()  #第一属性的种类数量

18

In [9]:
df2['Type 1'].value_counts()[: 3]  # 前三多数量对应的种类

Water     105
Normal     93
Grass      66
Name: Type 1, dtype: int64

In [160]:
# 分出单属性妖怪和多属性妖怪
df_type1 = df2[df2['Type 2'].isna()]
df_type2 = df2[df2['Type 2'].notna()]

In [161]:
df_type1

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43
10,8,Wartortle,Water,NaN,405,59,63,80,65,80,58
11,9,Blastoise,Water,NaN,530,79,83,100,85,105,78
...,...,...,...,...,...,...,...,...,...,...,...
775,705,Sliggoo,Dragon,NaN,452,68,75,53,83,113,60
776,706,Goodra,Dragon,NaN,600,90,100,70,110,150,80
788,712,Bergmite,Ice,NaN,304,55,69,85,32,35,28
789,713,Avalugg,Ice,NaN,514,95,117,184,44,46,28


In [162]:
df_type2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100
15,12,Butterfree,Bug,Flying,395,60,45,50,90,80,70
...,...,...,...,...,...,...,...,...,...,...,...
793,717,Yveltal,Dark,Flying,680,126,131,95,131,98,99
794,718,Zygarde50% Forme,Dragon,Ground,600,108,100,121,81,95,95
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70


In [13]:
df2.drop_duplicates(['Type 1', 'Type 2'])  # 第一属性和第二属性的组合种类

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43
13,10,Caterpie,Bug,NaN,195,45,30,35,20,20,45
...,...,...,...,...,...,...,...,...,...,...,...
773,703,Carbink,Rock,Fairy,500,50,50,150,50,150,50
778,708,Phantump,Ghost,Grass,309,43,70,48,50,60,38
790,714,Noibat,Flying,Dragon,245,40,30,35,45,40,55
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70


In [14]:
comb = df_type2.drop_duplicates(['Type 1', 'Type 2'])  # 具有两种属性的口袋妖怪，两种属性的组合

In [15]:
comb

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100
15,12,Butterfree,Bug,Flying,395,60,45,50,90,80,70
16,13,Weedle,Bug,Poison,195,40,35,30,20,20,50
20,16,Pidgey,Normal,Flying,251,40,45,40,35,35,56
...,...,...,...,...,...,...,...,...,...,...,...
773,703,Carbink,Rock,Fairy,500,50,50,150,50,150,50
778,708,Phantump,Ghost,Grass,309,43,70,48,50,60,38
790,714,Noibat,Flying,Dragon,245,40,30,35,45,40,55
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70


In [16]:
type1_1 = df2['Type 1'].drop_duplicates()
type2_1 = df2['Type 2'].drop_duplicates()

In [17]:
type1_2 = df_type2['Type 1'].drop_duplicates()
type2_2 = df_type2['Type 2'].drop_duplicates()

In [18]:
print(list(type1_1.sort_values()))
print(list(type1_2.sort_values()))

['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock', 'Steel', 'Water']
['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock', 'Steel', 'Water']


In [19]:
print(list(type2_1.sort_values()))
print(list(type2_2.sort_values()))

['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock', 'Steel', 'Water', nan]
['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock', 'Steel', 'Water']


In [20]:
print(list(type1_2.sort_values()))
print(list(type2_2.sort_values()))

['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock', 'Steel', 'Water']
['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock', 'Steel', 'Water']


In [21]:
print(list(type1_1.sort_values()))
print(list(type2_1.sort_values()))

['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock', 'Steel', 'Water']
['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock', 'Steel', 'Water', nan]


In [22]:
combs = comb[['Type 1', 'Type 2']].values

In [23]:
np.sort(combs)

array([['Grass', 'Poison'],
       ['Fire', 'Flying'],
       ['Bug', 'Flying'],
       ['Bug', 'Poison'],
       ['Flying', 'Normal'],
       ['Ground', 'Poison'],
       ['Fairy', 'Normal'],
       ['Flying', 'Poison'],
       ['Bug', 'Grass'],
       ['Fighting', 'Water'],
       ['Poison', 'Water'],
       ['Ground', 'Rock'],
       ['Psychic', 'Water'],
       ['Electric', 'Steel'],
       ['Ice', 'Water'],
       ['Ghost', 'Poison'],
       ['Grass', 'Psychic'],
       ['Ground', 'Rock'],
       ['Fairy', 'Psychic'],
       ['Ice', 'Psychic'],
       ['Flying', 'Water'],
       ['Rock', 'Water'],
       ['Flying', 'Rock'],
       ['Flying', 'Ice'],
       ['Electric', 'Flying'],
       ['Dragon', 'Flying'],
       ['Electric', 'Water'],
       ['Fairy', 'Flying'],
       ['Flying', 'Psychic'],
       ['Fairy', 'Water'],
       ['Flying', 'Grass'],
       ['Ground', 'Water'],
       ['Dark', 'Flying'],
       ['Normal', 'Psychic'],
       ['Bug', 'Steel'],
       ['Flying', 'Groun

In [24]:
combs.shape

(125, 2)

In [25]:
# 尚未出现过的属性组合
new_comb = []
lst = type1_2.values
for i in range(len(lst)):
    for j in range(i+1, len(lst)):
        if [lst[i], lst[j]] not in combs:
            new_comb.append([lst[i], lst[j]])

In [26]:
new_comb

[]

In [41]:
new_comb = []
full = []
lst = type1_2.values
for i in range(len(lst)):
    for j in range(i+1, len(lst)):
        full.append([lst[i], lst[j]])

In [42]:
d = pd.DataFrame(full)
d

,0,1
0,Grass,Fire
1,Grass,Bug
2,Grass,Normal
3,Grass,Poison
4,Grass,Water
...,...,...
148,Dark,Fighting
149,Dark,Flying
150,Steel,Fighting
151,Steel,Flying


In [59]:
d.drop_duplicates([0, 1])

,0,1
0,Grass,Fire
1,Grass,Bug
2,Grass,Normal
3,Grass,Poison
4,Grass,Water
...,...,...
148,Dark,Fighting
149,Dark,Flying
150,Steel,Fighting
151,Steel,Flying


参考答案

In [27]:
dp_dup = df.drop_duplicates('#', keep='first')
attr_dup = dp_dup.drop_duplicates(['Type 1', 'Type 2'])
attr_dup.shape[0]

143

In [28]:
L_full = [' '.join([i, j]) if i!=j else i for j in dp_dup['Type 1'].unique() for i in dp_dup['Type 1'].unique()]

In [29]:
L_full

['Grass',
 'Fire Grass',
 'Water Grass',
 'Bug Grass',
 'Normal Grass',
 'Poison Grass',
 'Electric Grass',
 'Ground Grass',
 'Fairy Grass',
 'Fighting Grass',
 'Psychic Grass',
 'Rock Grass',
 'Ghost Grass',
 'Ice Grass',
 'Dragon Grass',
 'Dark Grass',
 'Steel Grass',
 'Flying Grass',
 'Grass Fire',
 'Fire',
 'Water Fire',
 'Bug Fire',
 'Normal Fire',
 'Poison Fire',
 'Electric Fire',
 'Ground Fire',
 'Fairy Fire',
 'Fighting Fire',
 'Psychic Fire',
 'Rock Fire',
 'Ghost Fire',
 'Ice Fire',
 'Dragon Fire',
 'Dark Fire',
 'Steel Fire',
 'Flying Fire',
 'Grass Water',
 'Fire Water',
 'Water',
 'Bug Water',
 'Normal Water',
 'Poison Water',
 'Electric Water',
 'Ground Water',
 'Fairy Water',
 'Fighting Water',
 'Psychic Water',
 'Rock Water',
 'Ghost Water',
 'Ice Water',
 'Dragon Water',
 'Dark Water',
 'Steel Water',
 'Flying Water',
 'Grass Bug',
 'Fire Bug',
 'Water Bug',
 'Bug',
 'Normal Bug',
 'Poison Bug',
 'Electric Bug',
 'Ground Bug',
 'Fairy Bug',
 'Fighting Bug',
 'Psychic B

In [30]:
len(L_full)

324

In [31]:
L_part = [' '.join([i, j]) if type(j)!=float else i for i, j in zip(attr_dup['Type 1'], attr_dup['Type 2'])]

In [32]:
L_part

['Grass Poison',
 'Fire',
 'Fire Flying',
 'Water',
 'Bug',
 'Bug Flying',
 'Bug Poison',
 'Normal Flying',
 'Normal',
 'Poison',
 'Electric',
 'Ground',
 'Poison Ground',
 'Fairy',
 'Normal Fairy',
 'Poison Flying',
 'Bug Grass',
 'Fighting',
 'Water Fighting',
 'Psychic',
 'Water Poison',
 'Rock Ground',
 'Water Psychic',
 'Electric Steel',
 'Water Ice',
 'Ghost Poison',
 'Grass Psychic',
 'Ground Rock',
 'Grass',
 'Psychic Fairy',
 'Ice Psychic',
 'Water Flying',
 'Rock Water',
 'Rock Flying',
 'Ice Flying',
 'Electric Flying',
 'Dragon',
 'Dragon Flying',
 'Water Electric',
 'Fairy Flying',
 'Psychic Flying',
 'Water Fairy',
 'Rock',
 'Grass Flying',
 'Water Ground',
 'Dark',
 'Dark Flying',
 'Ghost',
 'Normal Psychic',
 'Bug Steel',
 'Ground Flying',
 'Steel Ground',
 'Bug Rock',
 'Bug Fighting',
 'Dark Ice',
 'Fire Rock',
 'Ice Ground',
 'Water Rock',
 'Steel Flying',
 'Dark Fire',
 'Water Dragon',
 'Rock Dark',
 'Psychic Grass',
 'Fire Fighting',
 'Water Grass',
 'Grass Dark',
 

In [33]:
len(L_part)

143

In [34]:
res = set(L_full).difference(set(L_part))

In [35]:
res

{'Bug Dark',
 'Bug Dragon',
 'Bug Fairy',
 'Bug Ice',
 'Bug Normal',
 'Bug Psychic',
 'Dark Bug',
 'Dark Electric',
 'Dark Fairy',
 'Dark Grass',
 'Dark Ground',
 'Dark Normal',
 'Dark Poison',
 'Dark Rock',
 'Dark Water',
 'Dragon Bug',
 'Dragon Dark',
 'Dragon Fairy',
 'Dragon Fighting',
 'Dragon Ghost',
 'Dragon Grass',
 'Dragon Normal',
 'Dragon Poison',
 'Dragon Rock',
 'Dragon Steel',
 'Dragon Water',
 'Electric Bug',
 'Electric Dark',
 'Electric Dragon',
 'Electric Fighting',
 'Electric Fire',
 'Electric Grass',
 'Electric Ground',
 'Electric Ice',
 'Electric Poison',
 'Electric Psychic',
 'Electric Rock',
 'Electric Water',
 'Fairy Bug',
 'Fairy Dark',
 'Fairy Dragon',
 'Fairy Electric',
 'Fairy Fighting',
 'Fairy Fire',
 'Fairy Ghost',
 'Fairy Grass',
 'Fairy Ground',
 'Fairy Ice',
 'Fairy Normal',
 'Fairy Poison',
 'Fairy Psychic',
 'Fairy Rock',
 'Fairy Steel',
 'Fairy Water',
 'Fighting Bug',
 'Fighting Dragon',
 'Fighting Electric',
 'Fighting Fairy',
 'Fighting Fire',
 'F

In [36]:
len(res)

181

In [37]:
combs2 = [' '.join(item) for item in combs]

In [38]:
combs2

['Grass Poison',
 'Fire Flying',
 'Bug Flying',
 'Bug Poison',
 'Normal Flying',
 'Poison Ground',
 'Normal Fairy',
 'Poison Flying',
 'Bug Grass',
 'Water Fighting',
 'Water Poison',
 'Rock Ground',
 'Water Psychic',
 'Electric Steel',
 'Water Ice',
 'Ghost Poison',
 'Grass Psychic',
 'Ground Rock',
 'Psychic Fairy',
 'Ice Psychic',
 'Water Flying',
 'Rock Water',
 'Rock Flying',
 'Ice Flying',
 'Electric Flying',
 'Dragon Flying',
 'Water Electric',
 'Fairy Flying',
 'Psychic Flying',
 'Water Fairy',
 'Grass Flying',
 'Water Ground',
 'Dark Flying',
 'Normal Psychic',
 'Bug Steel',
 'Ground Flying',
 'Steel Ground',
 'Bug Rock',
 'Bug Fighting',
 'Dark Ice',
 'Fire Rock',
 'Ice Ground',
 'Water Rock',
 'Steel Flying',
 'Dark Fire',
 'Water Dragon',
 'Rock Dark',
 'Psychic Grass',
 'Fire Fighting',
 'Water Grass',
 'Grass Dark',
 'Bug Water',
 'Grass Fighting',
 'Bug Ground',
 'Bug Ghost',
 'Dark Ghost',
 'Steel Fairy',
 'Steel Rock',
 'Fighting Psychic',
 'Water Dark',
 'Fire Ground'

In [39]:
len(combs2)

125

In [89]:
full

[['Grass', 'Fire'],
 ['Grass', 'Bug'],
 ['Grass', 'Normal'],
 ['Grass', 'Poison'],
 ['Grass', 'Water'],
 ['Grass', 'Rock'],
 ['Grass', 'Electric'],
 ['Grass', 'Ghost'],
 ['Grass', 'Ground'],
 ['Grass', 'Psychic'],
 ['Grass', 'Ice'],
 ['Grass', 'Dragon'],
 ['Grass', 'Fairy'],
 ['Grass', 'Dark'],
 ['Grass', 'Steel'],
 ['Grass', 'Fighting'],
 ['Grass', 'Flying'],
 ['Fire', 'Bug'],
 ['Fire', 'Normal'],
 ['Fire', 'Poison'],
 ['Fire', 'Water'],
 ['Fire', 'Rock'],
 ['Fire', 'Electric'],
 ['Fire', 'Ghost'],
 ['Fire', 'Ground'],
 ['Fire', 'Psychic'],
 ['Fire', 'Ice'],
 ['Fire', 'Dragon'],
 ['Fire', 'Fairy'],
 ['Fire', 'Dark'],
 ['Fire', 'Steel'],
 ['Fire', 'Fighting'],
 ['Fire', 'Flying'],
 ['Bug', 'Normal'],
 ['Bug', 'Poison'],
 ['Bug', 'Water'],
 ['Bug', 'Rock'],
 ['Bug', 'Electric'],
 ['Bug', 'Ghost'],
 ['Bug', 'Ground'],
 ['Bug', 'Psychic'],
 ['Bug', 'Ice'],
 ['Bug', 'Dragon'],
 ['Bug', 'Fairy'],
 ['Bug', 'Dark'],
 ['Bug', 'Steel'],
 ['Bug', 'Fighting'],
 ['Bug', 'Flying'],
 ['Normal', 'Poi

In [91]:
full_t = np.array(full[:])
for i in range(full_t.shape[0]):
    if full_t[i][0] > full_t[i][1]:
        tmp = full_t[i][0]
        full_t[i][0] = full_t[i][1]
        full_t[i][1] = tmp

In [92]:
test_td = pd.DataFrame(full_t)
test_td.sort_values(0)

,0,1
40,Bug,Psychic
36,Bug,Rock
35,Bug,Water
34,Bug,Poison
33,Bug,Normal
...,...,...
129,Psychic,Steel
79,Psychic,Water
95,Rock,Steel
75,Rock,Water


In [93]:
test_td.drop_duplicates([0, 1])

,0,1
0,Fire,Grass
1,Bug,Grass
2,Grass,Normal
3,Grass,Poison
4,Grass,Water
...,...,...
148,Dark,Fighting
149,Dark,Flying
150,Fighting,Steel
151,Flying,Steel


In [176]:
m = resu.to_numpy()

In [177]:
m = [ ' '.join(item) for item in m]

109

In [44]:
full2 = [' '.join(item) for item in full]
full2

['Grass Fire',
 'Grass Bug',
 'Grass Normal',
 'Grass Poison',
 'Grass Water',
 'Grass Rock',
 'Grass Electric',
 'Grass Ghost',
 'Grass Ground',
 'Grass Psychic',
 'Grass Ice',
 'Grass Dragon',
 'Grass Fairy',
 'Grass Dark',
 'Grass Steel',
 'Grass Fighting',
 'Grass Flying',
 'Fire Bug',
 'Fire Normal',
 'Fire Poison',
 'Fire Water',
 'Fire Rock',
 'Fire Electric',
 'Fire Ghost',
 'Fire Ground',
 'Fire Psychic',
 'Fire Ice',
 'Fire Dragon',
 'Fire Fairy',
 'Fire Dark',
 'Fire Steel',
 'Fire Fighting',
 'Fire Flying',
 'Bug Normal',
 'Bug Poison',
 'Bug Water',
 'Bug Rock',
 'Bug Electric',
 'Bug Ghost',
 'Bug Ground',
 'Bug Psychic',
 'Bug Ice',
 'Bug Dragon',
 'Bug Fairy',
 'Bug Dark',
 'Bug Steel',
 'Bug Fighting',
 'Bug Flying',
 'Normal Poison',
 'Normal Water',
 'Normal Rock',
 'Normal Electric',
 'Normal Ghost',
 'Normal Ground',
 'Normal Psychic',
 'Normal Ice',
 'Normal Dragon',
 'Normal Fairy',
 'Normal Dark',
 'Normal Steel',
 'Normal Fighting',
 'Normal Flying',
 'Poison W

In [60]:
len(combs2)

125

In [46]:
new_comb = []
for item in full2:
    if item not in combs2:
        new_comb.append(item)
        
print(new_comb)

['Grass Fire', 'Grass Bug', 'Grass Normal', 'Grass Water', 'Grass Rock', 'Grass Electric', 'Grass Ghost', 'Grass Dragon', 'Fire Bug', 'Fire Poison', 'Fire Electric', 'Fire Ghost', 'Fire Ice', 'Fire Dragon', 'Fire Fairy', 'Fire Dark', 'Bug Normal', 'Bug Psychic', 'Bug Ice', 'Bug Dragon', 'Bug Fairy', 'Bug Dark', 'Normal Poison', 'Normal Rock', 'Normal Electric', 'Normal Ghost', 'Normal Ice', 'Normal Dragon', 'Normal Dark', 'Normal Steel', 'Normal Fighting', 'Poison Rock', 'Poison Electric', 'Poison Ghost', 'Poison Psychic', 'Poison Ice', 'Poison Fairy', 'Poison Steel', 'Rock Electric', 'Rock Ghost', 'Electric Ground', 'Electric Psychic', 'Electric Ice', 'Electric Dragon', 'Electric Dark', 'Electric Fighting', 'Ghost Ground', 'Ghost Psychic', 'Ghost Ice', 'Ghost Fairy', 'Ghost Steel', 'Ghost Fighting', 'Ground Ice', 'Ground Fairy', 'Ground Fighting', 'Psychic Ice', 'Psychic Dragon', 'Psychic Dark', 'Psychic Steel', 'Ice Dragon', 'Ice Fairy', 'Ice Dark', 'Ice Steel', 'Ice Fighting', 'Drag

In [47]:
len(new_comb)

71

In [48]:
res = set(full2).difference(set(combs2))
len(res)

71

In [94]:
res2 = set(full2).intersection(set(combs2))
len(res2)

82

In [159]:
uni = set(full2).union(set(combs2))
len(uni)

196

In [183]:
len(m)

125

In [184]:
z = set(m).difference(set(full2))
z

{'Bug Fire',
 'Bug Grass',
 'Dark Dragon',
 'Dark Fire',
 'Dark Ghost',
 'Dark Grass',
 'Dark Ground',
 'Dark Ice',
 'Dark Poison',
 'Dark Psychic',
 'Dark Rock',
 'Dark Water',
 'Dragon Electric',
 'Dragon Fire',
 'Dragon Ghost',
 'Dragon Ground',
 'Dragon Ice',
 'Dragon Poison',
 'Dragon Psychic',
 'Dragon Rock',
 'Dragon Water',
 'Electric Normal',
 'Electric Water',
 'Fairy Grass',
 'Fairy Normal',
 'Fairy Psychic',
 'Fairy Rock',
 'Fairy Water',
 'Fighting Fire',
 'Fighting Grass',
 'Fighting Poison',
 'Fighting Psychic',
 'Fighting Rock',
 'Fighting Steel',
 'Fighting Water',
 'Flying Ghost',
 'Flying Grass',
 'Flying Ground',
 'Flying Ice',
 'Flying Normal',
 'Flying Poison',
 'Flying Psychic',
 'Flying Rock',
 'Flying Steel',
 'Flying Water',
 'Ghost Grass',
 'Ghost Poison',
 'Ghost Water',
 'Ground Normal',
 'Ground Poison',
 'Ground Rock',
 'Ground Water',
 'Ice Psychic',
 'Ice Rock',
 'Ice Water',
 'Psychic Rock',
 'Psychic Water',
 'Rock Water',
 'Steel Water'}

In [167]:
full2

['Grass Fire',
 'Grass Bug',
 'Grass Normal',
 'Grass Poison',
 'Grass Water',
 'Grass Rock',
 'Grass Electric',
 'Grass Ghost',
 'Grass Ground',
 'Grass Psychic',
 'Grass Ice',
 'Grass Dragon',
 'Grass Fairy',
 'Grass Dark',
 'Grass Steel',
 'Grass Fighting',
 'Grass Flying',
 'Fire Bug',
 'Fire Normal',
 'Fire Poison',
 'Fire Water',
 'Fire Rock',
 'Fire Electric',
 'Fire Ghost',
 'Fire Ground',
 'Fire Psychic',
 'Fire Ice',
 'Fire Dragon',
 'Fire Fairy',
 'Fire Dark',
 'Fire Steel',
 'Fire Fighting',
 'Fire Flying',
 'Bug Normal',
 'Bug Poison',
 'Bug Water',
 'Bug Rock',
 'Bug Electric',
 'Bug Ghost',
 'Bug Ground',
 'Bug Psychic',
 'Bug Ice',
 'Bug Dragon',
 'Bug Fairy',
 'Bug Dark',
 'Bug Steel',
 'Bug Fighting',
 'Bug Flying',
 'Normal Poison',
 'Normal Water',
 'Normal Rock',
 'Normal Electric',
 'Normal Ghost',
 'Normal Ground',
 'Normal Psychic',
 'Normal Ice',
 'Normal Dragon',
 'Normal Fairy',
 'Normal Dark',
 'Normal Steel',
 'Normal Fighting',
 'Normal Flying',
 'Poison W

In [146]:
test = comb[['Type 1', 'Type 2']]

In [147]:
test

,Type 1,Type 2
0,Grass,Poison
6,Fire,Flying
15,Bug,Flying
16,Bug,Poison
20,Normal,Flying
...,...,...
773,Rock,Fairy
778,Ghost,Grass
790,Flying,Dragon
797,Psychic,Ghost


In [148]:
test.drop_duplicates(['Type 1', 'Type 2'])

,Type 1,Type 2
0,Grass,Poison
6,Fire,Flying
15,Bug,Flying
16,Bug,Poison
20,Normal,Flying
...,...,...
773,Rock,Fairy
778,Ghost,Grass
790,Flying,Dragon
797,Psychic,Ghost


In [149]:
test2 = test.values

In [150]:
test2.shape

(125, 2)

In [151]:
for i in range(test2.shape[0]):
    if test2[i][0] > test2[i][1]:
        tmp = test2[i][0]
        test2[i][0] = test2[i][1]
        test2[i][1] = tmp

In [140]:
test2[120: 125]

array([['Rock', 'Fairy'],
       ['Ghost', 'Grass'],
       ['Flying', 'Dragon'],
       ['Psychic', 'Ghost'],
       ['Fire', 'Water']], dtype=object)

In [141]:

for i in range(120, 125):
    if test2[i][0] > test2[i][1]:
        tmp = test2[i][0]
        test2[i][0] = test2[i][1]
        test2[i][1] = tmp
        
print(test2[120: 125])

[['Fairy' 'Rock']
 ['Ghost' 'Grass']
 ['Dragon' 'Flying']
 ['Ghost' 'Psychic']
 ['Fire' 'Water']]


In [152]:
len(test2)

125

array([['Grass', 'Poison'],
       ['Fire', 'Flying'],
       ['Bug', 'Flying'],
       ['Bug', 'Poison'],
       ['Flying', 'Normal'],
       ['Ground', 'Poison'],
       ['Fairy', 'Normal'],
       ['Flying', 'Poison'],
       ['Bug', 'Grass'],
       ['Fighting', 'Water'],
       ['Poison', 'Water'],
       ['Ground', 'Rock'],
       ['Psychic', 'Water'],
       ['Electric', 'Steel'],
       ['Ice', 'Water'],
       ['Ghost', 'Poison'],
       ['Grass', 'Psychic'],
       ['Ground', 'Rock'],
       ['Fairy', 'Psychic'],
       ['Ice', 'Psychic'],
       ['Flying', 'Water'],
       ['Rock', 'Water'],
       ['Flying', 'Rock'],
       ['Flying', 'Ice'],
       ['Electric', 'Flying'],
       ['Dragon', 'Flying'],
       ['Electric', 'Water'],
       ['Fairy', 'Flying'],
       ['Flying', 'Psychic'],
       ['Fairy', 'Water'],
       ['Flying', 'Grass'],
       ['Ground', 'Water'],
       ['Dark', 'Flying'],
       ['Normal', 'Psychic'],
       ['Bug', 'Steel'],
       ['Flying', 'Groun

In [74]:
test3 = test2[:]
for i in range(test3.shape[0]):
    if test3[i][0] > test3[i][1]:
        tmp = test3[i][0]
        test3[i][0] = test3[i][1]
        test3[i][1] = tmp

In [75]:
test3

array([['Grass', 'Poison'],
       ['Fire', 'Flying'],
       ['Bug', 'Flying'],
       ['Bug', 'Poison'],
       ['Flying', 'Normal'],
       ['Ground', 'Poison'],
       ['Fairy', 'Normal'],
       ['Flying', 'Poison'],
       ['Bug', 'Grass'],
       ['Fighting', 'Water'],
       ['Poison', 'Water'],
       ['Ground', 'Rock'],
       ['Psychic', 'Water'],
       ['Electric', 'Steel'],
       ['Ice', 'Water'],
       ['Ghost', 'Poison'],
       ['Grass', 'Psychic'],
       ['Ground', 'Rock'],
       ['Fairy', 'Psychic'],
       ['Ice', 'Psychic'],
       ['Flying', 'Water'],
       ['Rock', 'Water'],
       ['Flying', 'Rock'],
       ['Flying', 'Ice'],
       ['Electric', 'Flying'],
       ['Dragon', 'Flying'],
       ['Electric', 'Water'],
       ['Fairy', 'Flying'],
       ['Flying', 'Psychic'],
       ['Fairy', 'Water'],
       ['Flying', 'Grass'],
       ['Ground', 'Water'],
       ['Dark', 'Flying'],
       ['Normal', 'Psychic'],
       ['Bug', 'Steel'],
       ['Flying', 'Groun

In [76]:
len(test3)

125

In [153]:
resu = pd.DataFrame(test2)
resu.sort_values(0)

,0,1
37,Bug,Rock
34,Bug,Steel
51,Bug,Water
101,Bug,Fire
65,Bug,Rock
...,...,...
72,Rock,Steel
57,Rock,Steel
21,Rock,Water
42,Rock,Water


In [154]:
resu.drop_duplicates([0, 1])

,0,1
0,Grass,Poison
1,Fire,Flying
2,Bug,Flying
3,Bug,Poison
4,Flying,Normal
...,...,...
119,Electric,Fairy
120,Fairy,Rock
121,Ghost,Grass
123,Ghost,Psychic


In [79]:
resu2 = pd.DataFrame(test3)
resu2.sort_values(0)

,0,1
37,Bug,Rock
34,Bug,Steel
51,Bug,Water
101,Bug,Fire
65,Bug,Rock
...,...,...
72,Rock,Steel
57,Rock,Steel
21,Rock,Water
42,Rock,Water


In [81]:
resu2.drop_duplicates([0, 1])

,0,1
0,Grass,Poison
1,Fire,Flying
2,Bug,Flying
3,Bug,Poison
4,Flying,Normal
...,...,...
119,Electric,Fairy
120,Fairy,Rock
121,Ghost,Grass
123,Ghost,Psychic


**3** 按照下述要求，构造`Series`：

* 取出物攻，超过120的替换为`high`，不足50的替换为`low`，否则设为`mid`
* 取出第一属性，分别用`replace`和`apply`替换所有字母为大写
* 求每个妖怪六项能力的离差，即所有能力中偏离中位数最大的值，添加到`df`并从大到小排序

In [195]:
df

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50
796,719,DiancieMega Diancie,Rock,Fairy,700,50,high,110,160,110,110
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,high,60,170,130,80


In [10]:
attac = df['Attack']
attac

0       49
1       62
2       82
3      100
4       52
      ... 
795    100
796    160
797    110
798    160
799    110
Name: Attack, Length: 800, dtype: int64

In [11]:
attac.mask(attac>120, 'high').mask(attac<50, 'low').mask((attac>=50) & (attac<=120), 'mid')

0       low
1       mid
2       mid
3       mid
4       mid
       ... 
795     mid
796    high
797     mid
798    high
799     mid
Name: Attack, Length: 800, dtype: object

In [9]:
attr1 = df['Type 1']

In [12]:
attr1.replace({i: str.upper(i) for i in attr1})

0        GRASS
1        GRASS
2        GRASS
3        GRASS
4         FIRE
        ...   
795       ROCK
796       ROCK
797    PSYCHIC
798    PSYCHIC
799       FIRE
Name: Type 1, Length: 800, dtype: object

In [15]:
attr1.apply(lambda tp: tp.upper())

0        GRASS
1        GRASS
2        GRASS
3        GRASS
4         FIRE
        ...   
795       ROCK
796       ROCK
797    PSYCHIC
798    PSYCHIC
799       FIRE
Name: Type 1, Length: 800, dtype: object

In [20]:
df['devi'] = df.iloc[:, 5:].max(1) - df.iloc[:, 5:].min(1)

In [23]:
df.sort_values('devi', ascending=False)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,devi
121,113,Chansey,Normal,NaN,450,250,5,5,35,105,50,245
261,242,Blissey,Normal,NaN,540,255,10,10,75,135,55,245
230,213,Shuckle,Bug,Rock,505,20,10,230,10,230,5,225
224,208,SteelixMega Steelix,Steel,Ground,610,75,125,230,55,95,30,200
333,306,AggronMega Aggron,Steel,NaN,630,70,140,230,60,80,50,180
...,...,...,...,...,...,...,...,...,...,...,...,...
396,362,Glalie,Ice,NaN,480,80,80,80,80,80,80,0
395,361,Snorunt,Ice,NaN,300,50,50,50,50,50,50,0
383,351,Castform,Normal,NaN,420,70,70,70,70,70,70,0
271,251,Celebi,Psychic,Grass,600,100,100,100,100,100,100,0


### Ex2：指数加权窗口
1. 作为扩张窗口的`ewm`窗口

在扩张窗口中，用户可以使用各类函数进行历史的累计指标统计，但这些内置的统计函数往往把窗口中的所有元素赋予了同样的权重。事实上，可以给出不同的权重来赋给窗口中的元素，指数加权窗口就是这样一种特殊的扩张窗口。

其中，最重要的参数是`alpha`，它决定了默认情况下的窗口权重为$w_i=(1−\alpha)^i,i\in\{0,1,...,t\}$，其中$i=t$表示当前元素，$i=0$表示序列的第一个元素。

从权重公式可以看出，离开当前值越远则权重越小，若记原序列为$x$，更新后的当前元素为$y_t$，此时通过加权公式归一化后可知：

$$
\begin{split}y_t &=\frac{\sum_{i=0}^{t} w_i x_{t-i}}{\sum_{i=0}^{t} w_i} \\
&=\frac{x_t + (1 - \alpha)x_{t-1} + (1 - \alpha)^2 x_{t-2} + ...
+ (1 - \alpha)^{t} x_{0}}{1 + (1 - \alpha) + (1 - \alpha)^2 + ...
+ (1 - \alpha)^{t}}\\\end{split}
$$

对于`Series`而言，可以用`ewm`对象如下计算指数平滑后的序列：

In [18]:
np.random.seed(0)
s = pd.Series(np.random.randint(-1,2,30).cumsum())
s.head()

0   -1
1   -1
2   -2
3   -2
4   -2
dtype: int32

In [19]:
s

0    -1
1    -1
2    -2
3    -2
4    -2
5    -1
6    -2
7    -1
8    -2
9    -3
10   -4
11   -3
12   -3
13   -2
14   -1
15   -2
16   -2
17   -2
18   -2
19   -2
20   -3
21   -3
22   -4
23   -5
24   -5
25   -4
26   -5
27   -4
28   -5
29   -5
dtype: int32

In [20]:
s.ewm(alpha=0.2).mean().head()

0   -1.000000
1   -1.000000
2   -1.409836
3   -1.609756
4   -1.725845
dtype: float64

In [24]:
s.shape[0]

30

In [21]:
s.ewm(alpha=0.2).mean()

0    -1.000000
1    -1.000000
2    -1.409836
3    -1.609756
4    -1.725845
5    -1.529101
6    -1.648273
7    -1.492481
8    -1.609720
9    -1.921223
10   -2.376048
11   -2.510047
12   -2.613738
13   -2.485343
14   -2.177441
15   -2.140925
16   -2.112091
17   -2.089261
18   -2.071148
19   -2.056753
20   -2.247158
21   -2.398846
22   -2.720978
23   -3.178945
24   -3.544537
25   -3.635906
26   -3.909386
27   -3.927544
28   -4.142368
29   -4.314107
dtype: float64

In [96]:
def expo_wind(s, a=0.2):
    # s, pandas series; a, 权重指数
    expos = [(1-a)**t for t in range(s.shape[0])]
    expos.reverse()
    return np.array((expos * s)).sum() / np.array(expos).sum()

In [98]:
(1-a) ** np.arange(s.shape[0])[::-1]

array([0.00154743, 0.00193428, 0.00241785, 0.00302231, 0.00377789,
       0.00472237, 0.00590296, 0.0073787 , 0.00922337, 0.01152922,
       0.01441152, 0.0180144 , 0.022518  , 0.0281475 , 0.03518437,
       0.04398047, 0.05497558, 0.06871948, 0.08589935, 0.10737418,
       0.13421773, 0.16777216, 0.2097152 , 0.262144  , 0.32768   ,
       0.4096    , 0.512     , 0.64      , 0.8       , 1.        ])

In [99]:
(1-a) ** np.arange(s.shape[0])

array([1.        , 0.8       , 0.64      , 0.512     , 0.4096    ,
       0.32768   , 0.262144  , 0.2097152 , 0.16777216, 0.13421773,
       0.10737418, 0.08589935, 0.06871948, 0.05497558, 0.04398047,
       0.03518437, 0.0281475 , 0.022518  , 0.0180144 , 0.01441152,
       0.01152922, 0.00922337, 0.0073787 , 0.00590296, 0.00472237,
       0.00377789, 0.00302231, 0.00241785, 0.00193428, 0.00154743])

In [79]:
s.expanding().apply(expo_wind)

0    -1.000000
1    -1.000000
2    -1.409836
3    -1.609756
4    -1.725845
5    -1.529101
6    -1.648273
7    -1.492481
8    -1.609720
9    -1.921223
10   -2.376048
11   -2.510047
12   -2.613738
13   -2.485343
14   -2.177441
15   -2.140925
16   -2.112091
17   -2.089261
18   -2.071148
19   -2.056753
20   -2.247158
21   -2.398846
22   -2.720978
23   -3.178945
24   -3.544537
25   -3.635906
26   -3.909386
27   -3.927544
28   -4.142368
29   -4.314107
dtype: float64

请用`expanding`窗口实现。

2. 作为滑动窗口的`ewm`窗口

从第1问中可以看到，`ewm`作为一种扩张窗口的特例，只能从序列的第一个元素开始加权。现在希望给定一个限制窗口`n`，只对包含自身的最近的`n`个元素作为窗口进行滑动加权平滑。请根据滑窗函数，给出新的`wi`与`yt`的更新公式，并通过`rolling`窗口实现这一功能。

$$
\begin{split}y_t &=\frac{\sum_{i=0}^{n-1} w_i x_{t-i}}{\sum_{i=0}^{n-1} w_i} \\
&=\frac{x_t + (1 - \alpha)x_{t-1} + (1 - \alpha)^2 x_{t-2} + ...
+ (1 - \alpha)^{n-1} x_{t-(n-1)}}{1 + (1 - \alpha) + (1 - \alpha)^2 + ...
+ (1 - \alpha)^{n-1}}\\\end{split}
$$

In [93]:
n = 4
roller = s.rolling(window=n)

In [94]:
def expo_wind(s, a=0.2, n=n):
    # s, pandas series; a, 权重指数; n, 滑动窗格长度
    expos = [(1-a)**t for t in range(s.shape[0])][: n]
    expos.reverse()
    s = s[: n]
    return np.array((expos * s)).sum() / np.array(expos).sum()

In [95]:
roller.apply(expo_wind)

0          NaN
1          NaN
2          NaN
3    -1.609756
4    -1.826558
5    -1.661247
6    -1.728997
7    -1.444444
8    -1.555556
9    -2.121951
10   -2.775068
11   -3.097561
12   -3.216802
13   -2.834688
14   -2.051491
15   -1.902439
16   -1.783198
17   -1.826558
18   -2.000000
19   -2.000000
20   -2.338753
21   -2.609756
22   -3.165312
23   -3.948509
24   -4.436314
25   -4.487805
26   -4.728997
27   -4.444444
28   -4.555556
29   -4.783198
dtype: float64